## Setup and Configuration


In [1]:
# Setup
import sys
import os
import json
import requests
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print("✅ Path setup complete")


Project root: /home/chris/github/AstrID
✅ Path setup complete


In [2]:
# Configuration
API_BASE = "http://127.0.0.1:8000"
MLFLOW_UI = "http://localhost:5000"
PREFECT_UI = "http://localhost:4200"

# Test parameters - Will be set after creating/loading survey
TEST_SURVEY_ID: Optional[str] = None
TEST_COUNT = 5  # Start small for testing
TEST_DATE_RANGE = {
    "start": "2024-01-01T00:00:00",
    "end": "2024-01-02T23:59:59"
}

# Authentication
AUTH_TOKEN: Optional[str] = None
AUTH_HEADERS: Dict[str, str] = {}

print(f"API Base: {API_BASE}")
print(f"Test Survey: {TEST_SURVEY_ID} (will be set after survey creation)")
print(f"Test Count: {TEST_COUNT}")
print("✅ Configuration complete")


API Base: http://127.0.0.1:8000
Test Survey: None (will be set after survey creation)
Test Count: 5
✅ Configuration complete


## 🔐 Authentication Cell (Copy to Other Notebooks)

**Copy the cell below to any notebook that needs API authentication:**


In [3]:
# from auth_helper import authenticate_user
# from src.core.constants import ADMIN_EMAIL, ADMIN_PASSWORD

# # Authenticate and build auth headers
# AUTH_TOKEN = authenticate_user(ADMIN_EMAIL, ADMIN_PASSWORD)
# AUTH_HEADERS = {
#     "Authorization": f"Bearer {AUTH_TOKEN}" if AUTH_TOKEN else "",
#     "Content-Type": "application/json",
# }


In [4]:
from src.core.constants import TRAINING_PIPELINE_API_KEY

global AUTH_HEADERS
AUTH_HEADERS = {
    "X-API-Key": TRAINING_PIPELINE_API_KEY,
    "Content-Type": "application/json",
}

In [5]:
display(TRAINING_PIPELINE_API_KEY)
display(AUTH_HEADERS)

'astrid__v2B8H-b6qRBfXpxNNNhAJFZcrbwIXUdnKvcloT5qgU'

{'X-API-Key': 'astrid__v2B8H-b6qRBfXpxNNNhAJFZcrbwIXUdnKvcloT5qgU',
 'Content-Type': 'application/json'}

## Survey Setup


In [6]:
import uuid

def create_test_survey() -> str | None:
    """Create a test survey for the pipeline."""
    if not AUTH_HEADERS:
        print("❌ Not authenticated. Please run the authentication cell first.")
        return None
    
    # Create a test survey using a direct database approach
    # Since there's no direct API endpoint, we'll use a known UUID
    # that should work with the system
    
    # Generate a proper UUID for the survey
    survey_id = str(uuid.uuid4())
    
    print(f"🔧 Creating test survey with ID: {survey_id}")
    print("   Note: This survey will be used for testing the pipeline")
    print("   In production, surveys should be created through the admin interface")
    
    return survey_id

# Set the survey ID
TEST_SURVEY_ID = create_test_survey()

if TEST_SURVEY_ID:
    print(f"🎯 Ready to use survey ID: {TEST_SURVEY_ID}")
    print(f"   This is a valid UUID format that the API will accept")
else:
    print("❌ Could not set up survey ID")


🔧 Creating test survey with ID: 85a67de7-709e-49d8-a04c-4e4f07c548a2
   Note: This survey will be used for testing the pipeline
   In production, surveys should be created through the admin interface
🎯 Ready to use survey ID: 85a67de7-709e-49d8-a04c-4e4f07c548a2
   This is a valid UUID format that the API will accept


## Step 1: Health Checks


In [7]:
def check_service_health(url: str, service_name: str) -> bool:
    """Check if a service is healthy."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✅ {service_name}: Healthy")
            return True
        else:
            print(f"❌ {service_name}: HTTP {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ {service_name}: {str(e)}")
        return False

# Check all services
services = {
    "API": f"{API_BASE}/health",
    "MLflow": MLFLOW_UI,
    "Prefect": PREFECT_UI
}

all_healthy = True
for name, url in services.items():
    if not check_service_health(url, name):
        all_healthy = False

if all_healthy:
    print("\n🎉 All services are healthy!")
else:
    print("\n⚠️  Some services are not healthy. Please check docker-compose.")
    print("Run: docker-compose up -d api worker prefect mlflow redis")


❌ API: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=5)
✅ MLflow: Healthy
✅ Prefect: Healthy

⚠️  Some services are not healthy. Please check docker-compose.
Run: docker-compose up -d api worker prefect mlflow redis


## Step 2: Ingest Test Observations


In [8]:
def ingest_test_observations() -> Dict[str, Any]:
    """Ingest test observations using the API."""
    # AUTH_HEADERS should already be set globally in cell 6
    if not AUTH_HEADERS:
        print("❌ AUTH_HEADERS not set. Please run the authentication cell first.")
        return {"success": False, "error": "AUTH_HEADERS not set. Please run the authentication cell first."}
    
    url = f"{API_BASE}/observations/ingest/batch-random"
    payload = {
        "count": TEST_COUNT,
        "survey_id": TEST_SURVEY_ID
    }
    
    print(f"Ingesting {TEST_COUNT} test observations for survey {TEST_SURVEY_ID}...")
    
    try:
        response = requests.post(url, json=payload, headers=AUTH_HEADERS, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        observations = result.get("data", [])
        
        print(f"✅ Successfully ingested {len(observations)} observations")
        
        # Store observation IDs for later use
        observation_ids = [obs["id"] for obs in observations]
        
        return {
            "success": True,
            "count": len(observations),
            "observation_ids": observation_ids,
            "observations": observations
        }
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to ingest observations: {str(e)}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                error_detail = e.response.json()
                print(f"   Error details: {error_detail}")
            except:
                print(f"   Response: {e.response.text}")
        return {"success": False, "error": str(e)}

# Run ingestion
ingestion_result = ingest_test_observations()

if ingestion_result["success"]:
    print(f"\nObservation IDs: {ingestion_result['observation_ids']}")
    # Store for later steps
    observation_ids = ingestion_result["observation_ids"]
else:
    print("\n❌ Cannot proceed without observations")
    observation_ids = []


Ingesting 5 test observations for survey 85a67de7-709e-49d8-a04c-4e4f07c548a2...
✅ Successfully ingested 0 observations

Observation IDs: []


In [9]:
def create_training_dataset() -> Dict[str, Any]:
    """Create training dataset from the processed detections."""
    url = f"{API_BASE}/training/datasets/collect"
    payload = {
        "survey_ids": [TEST_SURVEY_ID],
        "start": TEST_DATE_RANGE["start"],
        "end": TEST_DATE_RANGE["end"],
        "confidence_threshold": 0.3,  # Lower threshold for testing
        "max_samples": 100,
        "name": f"test_{TEST_SURVEY_ID}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    }
    
    print(f"Creating training dataset for survey {TEST_SURVEY_ID}...")
    print(f"Date range: {payload['start']} to {payload['end']}")
    print(f"Confidence threshold: {payload['confidence_threshold']}")
    
    try:
        response = requests.post(url, json=payload, headers=AUTH_HEADERS, timeout=60)
        response.raise_for_status()
        
        result = response.json()
        dataset_info = result.get("data", {})
        
        print(f"✅ Successfully created training dataset")
        print(f"   - Dataset ID: {dataset_info.get('dataset_id')}")
        print(f"   - Name: {dataset_info.get('name')}")
        print(f"   - Total samples: {dataset_info.get('total')}")
        print(f"   - Quality score: {dataset_info.get('quality', {}).get('quality_score', 0):.3f}")
        
        return {
            "success": True,
            "dataset_id": dataset_info.get("dataset_id"),
            "dataset_info": dataset_info
        }
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to create training dataset: {str(e)}")
        return {"success": False, "error": str(e)}

# Create training dataset
dataset_result = create_training_dataset()

if dataset_result["success"]:
    dataset_id = dataset_result["dataset_id"]
    print(f"\n🎉 Training dataset created successfully!")
    print(f"Dataset ID: {dataset_id}")
else:
    print(f"\n❌ Training dataset creation failed: {dataset_result['error']}")
    dataset_id = None


Creating training dataset for survey 85a67de7-709e-49d8-a04c-4e4f07c548a2...
Date range: 2024-01-01T00:00:00 to 2024-01-02T23:59:59
Confidence threshold: 0.3
✅ Successfully created training dataset
   - Dataset ID: 0acda9eb-a45d-448a-a434-ae1806f2bb99
   - Name: test_85a67de7-709e-49d8-a04c-4e4f07c548a2_20250922_234403
   - Total samples: 0
   - Quality score: 0.000

🎉 Training dataset created successfully!
Dataset ID: 0acda9eb-a45d-448a-a434-ae1806f2bb99


In [10]:
def list_training_datasets() -> Dict[str, Any]:
    """List all available training datasets."""
    url = f"{API_BASE}/training/datasets"
    
    try:
        response = requests.get(url, headers=AUTH_HEADERS, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        datasets = result.get("data", [])
        
        print(f"✅ Found {len(datasets)} training datasets")
        
        for i, dataset in enumerate(datasets, 1):
            print(f"\n{i}. {dataset.get('name')}")
            print(f"   - ID: {dataset.get('id')}")
            print(f"   - Samples: {dataset.get('total_samples')}")
            print(f"   - Quality: {dataset.get('quality_score', 0):.3f}")
            print(f"   - Status: {dataset.get('status')}")
            print(f"   - Created: {dataset.get('created_at')}")
        
        return {
            "success": True,
            "datasets": datasets
        }
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to list datasets: {str(e)}")
        return {"success": False, "error": str(e)}

# List all datasets
list_result = list_training_datasets()

if list_result["success"]:
    datasets = list_result["datasets"]
    
    # Find datasets with samples
    datasets_with_samples = [d for d in datasets if d.get("total_samples", 0) > 0]
    
    if datasets_with_samples:
        print(f"\n🎉 Found {len(datasets_with_samples)} datasets with samples!")
        print(f"\nReady for training:")
        for dataset in datasets_with_samples:
            print(f"- {dataset['name']} (ID: {dataset['id']}) - {dataset['total_samples']} samples")
    else:
        print(f"\n⚠️  No datasets have samples yet.")
        print(f"This suggests the data pipeline needs to be run first.")
else:
    print(f"\n❌ Failed to list datasets: {list_result['error']}")


✅ Found 11 training datasets

1. test_85a67de7-709e-49d8-a04c-4e4f07c548a2_20250922_234403
   - ID: 0acda9eb-a45d-448a-a434-ae1806f2bb99
   - Samples: 0
   - Quality: 0.000
   - Status: active
   - Created: 2025-09-23T06:44:04.662205+00:00

2. test_dad7665f-907b-4bcd-bf77-237501714edf_20250922_220709
   - ID: 7d26d30a-3af5-4597-b5a2-3bbb5c3f1b8d
   - Samples: 0
   - Quality: 0.000
   - Status: active
   - Created: 2025-09-23T05:07:09.989516+00:00

3. smoketest_hst_2024
   - ID: 732d7e5d-f75d-42d1-8e30-0f1b4c5fd46a
   - Samples: 0
   - Quality: 0.000
   - Status: active
   - Created: 2025-09-23T04:52:04.801897+00:00

4. test_f1eaed84-6366-4e22-8a0e-4cda1b699aa8_20250922_215005
   - ID: e7b7abb1-febe-4f29-83a9-a887f4c4aec1
   - Samples: 0
   - Quality: 0.000
   - Status: active
   - Created: 2025-09-23T04:50:06.751772+00:00

5. test_hst_20250922_172345
   - ID: 0a0dc4f1-f318-4287-a9a7-6fa1537d425b
   - Samples: 0
   - Quality: 0.000
   - Status: active
   - Created: 2025-09-23T00:23:45.4